In [7]:
import numpy as np
import matplotlib.pyplot as plt

In [22]:
# Set previous variables 
t_max = 150e-3   # second
dt = 1e-3        # second
tau = 20e-3      # second
el = -60e-3      # milivolt
vr = -70e-3      # milivolt
vth = -50e-3     # milivolt
r = 100e6        # ohm
i_mean = 25e-11  # ampere


0.149
0.001


### Coding Exercise 1: Plotting a histogram

In [18]:
# Set a random number generator
np.random.seed(2020)

# Initialize t_range, step_end, n, v_n, i, and nbins
t_range = np.arange(0, t_max, dt)
step_end = len(t_range)
n = 10000
v_n = 
i = 
nbins = 50

print(len(t_range))

150
